<a href="https://colab.research.google.com/github/sucrammal/vectari/blob/main/COSTAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [22]:
!pip install tqdm

In [64]:
!pip install ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.8 MB/s eta 0:00:00


In [65]:
import pandas as pd
import openai
from openai import AzureOpenAI
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split

import ipdb

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import *

nltk.download('punkt')
nltk.download('stopwords')

from google.colab import drive
drive.mount('/content/drive')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from google.colab import userdata

# Setting up API
gpt_config = {
    'model': 'gpt-4-vectari-1106preview',
    'azure_endpoint': 'https://oai-vectari-eastus2.openai.azure.com/',
    'api_key': userdata.get('openAPIKey'),
    'api_version': '2023-12-01-preview'
}

# gpt_config = {
#     'model': 'gpt-35-turbo',
#     'azure_endpoint': 'https://oai-vectari-northcentral-us.openai.azure.com/',
#     'api_key': userdata.get('openAPIKey3.5'),
#     'api_version': '2024-02-15-preview'
# }

print(gpt_config['azure_endpoint'])

client = AzureOpenAI(
  azure_endpoint = gpt_config['azure_endpoint'],
  api_key = gpt_config['api_key'],
  api_version = gpt_config['api_version']
)

https://oai-vectari-eastus2.openai.azure.com/


**Load data**

In [ ]:
# instantiate input file from scratch.
input_file = input("Enter file path of raw dataset")
with open(input_file, 'r') as file:
  all_complaints = pd.read_csv(file)

# extract complaints with transcripts
complaints_with_transcripts = all_complaints.loc[all_complaints["Consumer complaint narrative"].notna()]
complaints_with_transcripts.shape[0]

In [37]:
# load LLM pre-cleaned transcripts from csv
input_file = input("Enter a file name that exists in the Data folder: ")
cleaned_dataset = pd.DataFrame()

with open(input_file, 'r') as file:
    cleaned_dataset = pd.read_csv(file)

Enter a file name that exists in the Data folder: /content/drive/MyDrive/Internships and Work/Summer 2024/Vectari/out/LLM_cleaned.csv


**This function uses the LLM to clean incoming transcripts**

In [10]:
cleaning_system = """
CONTEXT: You will receive a transcript of a phone call between a call center agent at a mortgage servicing company and a customer/borrower. You need to clean this transcript.

#############

OBJECTIVE: Your first job is to clean the incoming low-quality transcripts from the first dataset. These are calls about financial services and we have seen some text completely out of place.  For instance 'the first drug was fixed, and any drug after that was gonna be viral' should pretty clearly be
'the first rate was fixed, and any rate after that was going to be variable'.
Your job is to parse the text and using your natural language understanding and contextual awareness, please re-write the transcript and clean any grammar / things that do not make sense in financial use cases.
Additionally, remove any unwanted 'x's, 'xxxx's, any other similar phrases representing redaction, and simply any text or numbers that aren't words. Remove any dates, names, and otherwise sensitive information. After the response is generated, remove all full stops, commas, semicolons, colons, and quotation marks.

#############

# STYLE #
Financial complaint transcript

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: A PARAGRAPH OF TEXT #
"""

**Then, applying the cleaning function, ask the LLM to cluster the data**

In [11]:
def clean_transcript(transcript):
  messages = [
    {
        "role":"system",
        "content": cleaning_system
    },
    {
        "role":"user",
        "content":f'"""{transcript}"""'
    }
  ]

  response = client.chat.completions.create(
      model=gpt_config['model'],
      messages = messages
  )

  LLM_cleaned_sample = response.choices[0].message.content
  return LLM_cleaned_sample

In [83]:
cluster_system = """
CONTEXT: You will now be recieving a large text containing transcripts of phone calls between a call center agent at a mortgage servicing company and a customer/borrower. Individual transcripts
are separated by two newline characters, "\n\n", from one another. This large text represents a CLUSTER of transcripts that have been categorized based on content under some label, for example, "fraud or scam".

#############

OBJECTIVE:
Based on the cluster you have received,
1. CLUSTER_INFORMATION: Describe the cluster in terms of the dataset columns.
2. CLUSTER_CATEGORY_NAME: Interpret [CLUSTER_INFORMATION] to obtain a short name for the category of the transcripts in this cluster.

#############

# STYLE #
Business analytics report

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: MARKDOWN REPORT #
<From processing the CLUSTER you have made>
– Complaint Group: [CLUSTER_CATEGORY_NAME]
— Complaint Group Profile: [CLUSTER_INFORMATION]

"""

In [78]:
def cluster_complaints(cleaned_dataset):
    clusters = cleaned_dataset.groupby('Issue')
    cluster_info = {}

    for label, group in tqdm(clusters, desc="Creating clusters"):
        # Collect all transcripts in this cluster
        transcripts = group['Consumer complaint narrative'].tolist()

        # Weird error. Passing the full, unsplit dataset through this function worked.
        # But, passing a train set (from train_test_split) would crash since a value in transcripts would somehow not be a string.
        filtered_list = [s for s in transcripts if isinstance(s, str)]

        # Extract keywords and name cluster group.
        all_transcripts_joined = "\n\n".join(filtered_list)


        messages = [
          {
              "role":"system",
              "content": cluster_system
          },
          {
              "role":"user",
              "content":f'"""{all_transcripts_joined}"""'
          }
        ]

        response = client.chat.completions.create(
            model=gpt_config['model'],
            messages = messages
        )

        cluster_report = response.choices[0].message.content


        cluster_info[label] = cluster_report

    return cluster_info

In [14]:
classify_system = """
CONTEXT: You have previously created Complaint Groups of transcripts of complaint phone calls between a call center agent at a mortgage servicing company and a customer/borrower based
on the category of the complaints. Transcripts under each Complaint Group has been categorized based on content under some label, for example, "fraud or scam".

You will now be recieving two things: the first is a set of markdown reports representing the information you have gathered and organized for each CLUSTER, and are of the following format:
"– Complaint Group: [CLUSTER_CATEGORY_NAME]
— Complaint Group Profile: [CLUSTER_INFORMATION]"
Each markdown report for each Complaint Group will be separated by a new line character "\n".

The second piece of data you will receive is a transcript that has not been categorized into one of your CLUSTERs yet.

#############

OBJECTIVE: Based on your natural language understanding, and your understanding of the read and determine which Complaint Group, the incoming transcript belongs under.

#############

# STYLE #
Business analytics report

#############

# TONE #
Professional, technical

#############

# AUDIENCE #
Banks. I want accurate classification so that they will adopt this methodology to help organize their complaints database.

#############

# RESPONSE: Single line of text containing ONLY one [CLUSTER_CATEGORY_NAME] of your choosing. #
[CLUSTER_CATEGORY_NAME]
"""

In [84]:
def classify_transcript(cluster_info, new_transcript):

  # unpack and format cluster_info, aka the reports
  markdown_reports = ""

  for report in cluster_info.values():
    markdown_reports = "\n".join(report)

  messages = [
          {
              "role":"system",
              "content": classify_system
          },
          {
              "role":"user",
              "content":f'"""{markdown_reports} \n {new_transcript}"""'
          }
        ]

  response = client.chat.completions.create(
            model=gpt_config['model'],
            messages = messages
        )

  complaint_label = response.choices[0].message.content

  return complaint_label


In [ ]:
# Run this to clean transcipts from scratch.
LLM_cleaned_sentences = []
all_labels = complaints_with_transcripts['Issue']

# First, LLM cleanup
tqdm.pandas(desc="LLM transcript cleaning")
LLM_cleaned_sentences = complaints_with_transcripts["Consumer complaint narrative"].progress_apply(lambda x : clean_transcript(x))

cleaned_dataset = pd.DataFrame({'Consumer complaint narrative': LLM_cleaned_sentences, 'Issue': all_labels})

**Train-test split before clustering**

In [79]:
train_dataset, test_dataset = train_test_split(cleaned_dataset, random_state=104, test_size=0.20, shuffle = True)

In [80]:
cluster_info = cluster_complaints(train_dataset)

Creating clusters: 100%|██████████| 6/6 [03:10<00:00, 31.79s/it]


In [81]:
print(cluster_info)

{'Fraud or scam': '# RESPONSE: MARKDOWN REPORT #\n\n<From processing the CLUSTER you have received>:\n– Complaint Group: **Fraudulent Money Transfers and Scams**\n— Complaint Group Profile: Instances involving customers who have been victimized by fraudulent money transfers and scams through practices like false promises of services, misleading sales tactics, impersonation, hacking, and unauthorized changes in money transfer recipients. Victims often report transactions via wire services like Western Union or MoneyGram, or through services like PayPal, where funds were sent but services or products were never received, and refunds were denied or unattainable. Some complaints involve complex interactions with financial institutions where the legitimacy of transactions and communication was questionable, leading to significant monetary losses and subsequent difficulties in recovering these funds.\n\n', 'Incorrect/missing disclosures or info': "### Complaint Group: Payment Transfer Issues

In [89]:
# Optional: export cluster data to a file.
import json

# Convert dictionary to a string
dict_string = json.dumps(cluster_info, indent=4)

# Write the string to a file
with open('cluster_info.txt', 'w') as file:
    file.write(dict_string)

!cp cluster_info.txt "/content/drive/MyDrive/Internships and Work/Summer 2024/Vectari/out"


In [86]:
print(test_dataset.iloc[0])
classify_transcript(cluster_info, test_dataset.iloc[0]['Consumer complaint narrative'])

Consumer complaint narrative    On XXXX/XXXX/15, I initiated a money transfer ...
Issue                                                    Other transaction issues
Name: 963, dtype: object


'Transaction and Transfer Issues'